## Steps 
1.Screen Capture

2.Generate Dataset and load it
   
3.Fit the model using SVC and calculate accuracy 

4.Prediction of image drawn in paint

## 1.Screen Capture

In [30]:
def one_time():
    import pyscreenshot as ImageGrab
    import time
    images_folder = r"C:\Users\prade\OneDrive\Documents\ML Project\Captured_images\9"  # Use r for raw string

    for i in range(5):
        time.sleep(8)
        im = ImageGrab.grab(bbox=(60, 350, 400, 550))  # x1, y1, x2, y2
        im.save(f"{images_folder}\\{i}.png")  # Save the image with double backslash
        print(f"Saved image {i}.png. Clear screen now and redraw.")


Saved image 0.png. Clear screen now and redraw.
Saved image 1.png. Clear screen now and redraw.
Saved image 2.png. Clear screen now and redraw.
Saved image 3.png. Clear screen now and redraw.
Saved image 4.png. Clear screen now and redraw.


## 2.Generate Dataset

In [30]:
import cv2
import csv
import glob
header =["label"]
for i in range(0,784):
    header.append("pixel"+str(i))
with open('dataset.csv','a') as f:
    writer=csv.writer(f)
    writer.writerow(header)
for label in range(10):
    dirList = glob.glob(f"C:\\Users\\prade\\OneDrive\\Documents\\ML Project\\Captured_images\\{label}\\*.png")
    for img_path in dirList:
        im = cv2.imread(img_path)
        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im_gray = cv2.GaussianBlur(im_gray, (15, 15), 0)
        roi = cv2.resize(im_gray, (28, 28), interpolation=cv2.INTER_AREA)
        data = [label]  # Start the data list with the label
        rows, cols = roi.shape

        # Add pixel one by one into data array
        for i in range(rows):
            for j in range(cols):
                k = roi[i, j]
                data.append(1 if k > 100 else 0)  # More concise pixel assignment

        # Write the row to the CSV file
        with open('dataset.csv', 'a', newline='') as f:  # Use newline=''
            writer = csv.writer(f)
            writer.writerow(data)  # Append the correct data, not header


## Loading the dataset

In [31]:
import pandas as pd 
from sklearn.utils import shuffle 
#0,....,1.....,2.....
#5,3,1,0,2,5,.......
 
data = pd.read_csv('dataset.csv')
data = shuffle(data)
print(data)

    label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
93      8       0       0       0       0       0       0       0       0   
89      7       0       0       0       0       0       0       0       0   
75      4       0       0       0       0       0       0       0       0   
23  label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7   
86      6       0       0       0       0       0       0       0       0   
..    ...     ...     ...     ...     ...     ...     ...     ...     ...   
40  label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7   
6   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7   
1   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7   
95      8       0       0       0       0       0       0       0       0   
14  label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7   

    pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779

## Separation of Dependent and Independent Variavle

In [32]:
X=data.drop(["label"],axis=1)
Y=data["label"]

### Preview of one image using Matplotlib

In [33]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
idx = 6
img = X.iloc[idx].values.reshape(28,28)  # Reshaping a 784-length array into a 28x28 image
print(Y.iloc[idx])  # Printing the label for the image


4


## Train-Test Split 

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import joblib
import os

# Assuming X and Y are already defined as your features and labels
# Convert categorical variables to numeric using one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets with stratification
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=6, stratify=Y)


## 3.Fit the model using SVC and calculate accuracy

### Fit the model using SVC and also to save the model using joblib.

In [41]:
# Check unique classes in train_y
#print("Unique classes in train_y:", train_y.unique())
#print("Number of unique classes in train_y:", len(train_y.unique()))

# Check if the model directory exists, if not create it
if not os.path.exists('model'):
    os.makedirs('model')

# Initialize the Support Vector Classifier with a linear kernel
classifier = SVC(kernel="linear", random_state=6)

# Train the classifier
classifier.fit(train_x, train_y)

# Save the trained model using joblib
joblib.dump(classifier, "model/digit_recognizer.pkl")

#Test the model to ensure it was trained properly
accuracy = classifier.score(test_x, test_y)
#print(f"Model accuracy: {accuracy:.4f}")
# Print the entire target variable Y

from sklearn import metrics 
prediction =classifier.predict(test_x)
print("Accuracy =" ,metrics.accuracy_score(prediction,test_y))
#print("Target variable Y:", Y)
#print("Unique classes in Y:", Y.unique())
#print("Number of unique classes in Y:", len(Y.unique()))
from sklearn.model_selection import cross_val_score

scores = cross_val_score(classifier, X, Y, cv=5)
print("Cross-validated scores:", scores)
print("Mean accuracy:", scores.mean())



Accuracy = 0.7142857142857143
Cross-validated scores: [0.9047619  0.85714286 0.7        0.8        0.7       ]
Mean accuracy: 0.7923809523809524


### Prediction of image drawn in Paint

In [ ]:
import joblib
import cv2
import numpy as np
import time
import pyscreenshot as ImageGrab
import os

# Load the pre-trained model
model = joblib.load(r"C:\Users\prade\OneDrive\Documents\ML Project\model\digit_recognizer")

# Specify the image folder and ensure it exists
images_folder = r"C:\Users\prade\OneDrive\Documents\ML Project\img\\"
if not os.path.exists(images_folder):
    os.makedirs(images_folder)

while True:
    # Capture the screen within the specified coordinates (bbox)
    img = ImageGrab.grab(bbox=(60, 350, 400, 550))
    
    # Save the captured image in the designated folder
    img.save(images_folder + "img.png")
    
    # Load the saved image and process it using OpenCV
    im = cv2.imread(images_folder + "img.png")
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gray = cv2.GaussianBlur(im_gray, (15, 15), 0)

    # Apply binary thresholding to the image
    ret, im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
    
    # Resize the image to 28x28 pixels
    roi = cv2.resize(im_th, (28, 28), interpolation=cv2.INTER_AREA)

    rows, cols = roi.shape
    X = []

    # Convert the image to a 1D array of pixel values
    for i in range(rows):
        for j in range(cols):
            k = roi[i, j]
            X.append(1 if k > 100 else 0)

    # Use the loaded model to make predictions
    predictions = model.predict([X])
    print("Prediction:", predictions[0])
    
    # Display the prediction on the image using OpenCV
    cv2.putText(im, "Prediction: " + str(predictions[0]), (20, 20), 0, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Show the processed image with the prediction
    cv2.startWindowThread()
    cv2.namedWindow("Result")
    cv2.imshow("Result", im)
    
    # Wait for a key press; break the loop if 'Enter' is pressed
    if cv2.waitKey(1) == 13:  # ASCII value 13 is 'Enter'
        break

# Close all OpenCV windows
cv2.destroyAllWindows()
